In [10]:
import numpy as np
from sympy import *
from scipy.spatial.transform import Rotation

alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha1:7')
a1, a2, a3, a4, a5, a6 = symbols('a1:7')
d1, d2, d3, d4, d5, d6 = symbols('d1:7')
q1, q2, q3, q4, q5, q6 = symbols('q1:7')


def set_dh_table():
    """

    Returns: dictionary

    Important: all length arguments in [m]
               all angles argument in [radians]

    """

    dh_subs_dict = {alpha1: pi / 2, a1: 0, d1: 0.2433, q1: q1,
                    alpha2: pi, a2: 0.28, d2: 0.03, q2: q2 + pi / 2,
                    alpha3: pi / 2, a3: 0, d3: 0.02, q3: q3 + pi / 2,
                    alpha4: pi / 2, a4: 0, d4: 0.245, q4: q4 + pi / 2,
                    alpha5: pi / 2, a5: 0, d5: 0.057, q5: q5 + pi,
                    alpha6: 0, a6: 0, d6: 0.235, q6: q6 + pi / 2}
    return dh_subs_dict

    


def dh(alpha, a, d, theta):
    """
    Args:
        alpha: torsion angle
        a: distance
        d: translation
        theta: rotation angle

    Returns: Homogeneous DH matrix

    Important note: use sympy cos/sin arguments instead of math/numpy versions.
    i.e: cos(theta) \ sin(theta)

    """
    # Formula
    return Matrix([[cos(theta), -cos(alpha)*sin(theta), sin(alpha)*sin(theta), a*cos(theta)],
                     [sin(theta), cos(alpha)*cos(theta), -sin(alpha)*cos(theta), a*sin(theta)],
                     [0, sin(alpha), cos(alpha), d],
                     [0, 0, 0, 1]])               

    

def transform_matrices():

    dictionary = set_dh_table()
    T_01 = dh(dictionary[alpha1], dictionary[a1], dictionary[d1], dictionary[q1])
    T_12 = dh(dictionary[alpha2], dictionary[a2], dictionary[d2], dictionary[q2])
    T_23 = dh(dictionary[alpha3], dictionary[a3], dictionary[d3], dictionary[q3])
    T_34 = dh(dictionary[alpha4], dictionary[a4], dictionary[d4], dictionary[q4])
    T_45 = dh(dictionary[alpha5], dictionary[a5], dictionary[d5], dictionary[q5])
    T_56 = dh(dictionary[alpha6], dictionary[a6], dictionary[d6], dictionary[q6])
    Tes = Matrix([[0, -1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
    T = T_01 * T_12 * T_23 * T_34 * T_45 * T_56 * Tes

    return T


def FK(theta_list):

    """
    Args:
        theta_list: joint angle vector ---> list [1,6]
    Returns:
        End effector homogeneous matrix --> Matrix((4, 4))

    Hints:
        - we have added a sympy implementation with missing parts, u dont have to use the same method.
        - chain 'Tes' to T_06 at the end.
    """

    dictionary = set_dh_table()
    T_01 = dh(dictionary[alpha1], dictionary[a1], dictionary[d1], dictionary[q1])
    T_12 = dh(dictionary[alpha2], dictionary[a2], dictionary[d2], dictionary[q2])
    T_23 = dh(dictionary[alpha3], dictionary[a3], dictionary[d3], dictionary[q3])
    T_34 = dh(dictionary[alpha4], dictionary[a4], dictionary[d4], dictionary[q4])
    T_45 = dh(dictionary[alpha5], dictionary[a5], dictionary[d5], dictionary[q5])
    T_56 = dh(dictionary[alpha6], dictionary[a6], dictionary[d6], dictionary[q6])
    Tes = Matrix([[0, -1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
    T = T_01 * T_12 * T_23 * T_34 * T_45 * T_56 * Tes
    #print(T)
    ''' fill angles to dict for sympy calculations'''

    theta_dict = {q1:theta_list[0], q2:theta_list[1], q3:theta_list[2], q4:theta_list[3], q5:theta_list[4], q6:theta_list[5]}
    # for i in range(len(theta_list)):
    #     theta_dict[q[i]] = theta_list[i]
    
    ''' 
    homogeneous transformation matrix from base_link to end_effector [type: numeric matrix] 
    because we are using sympy, we have to use evalf.
    '''
    T_0G_eval = T.evalf(subs=theta_dict, chop=True, maxn=4)

    return T_0G_eval

    

def xyz_euler(A):
    """
    Extract translation and orientation in euler angles

    Args:
        A: Homogeneous transformation --> np.array((4, 4))

    Returns: x, y, z, thetax, thetay, thetaz --> np.array((1, 6))

    Important note: use numpy arrays

    """
    
    A00 = np.array(A[0, 0]).astype(np.float32)
    A01 = np.array(A[0, 1]).astype(np.float32)
    A02 = np.array(A[0, 2]).astype(np.float32)
    A03 = np.array(A[0, 3]).astype(np.float32)
    A10 = np.array(A[1, 0]).astype(np.float32)
    A11 = np.array(A[1, 1]).astype(np.float32)
    A12 = np.array(A[1, 2]).astype(np.float32)
    A13 = np.array(A[1, 3]).astype(np.float32)
    A20 = np.array(A[2, 0]).astype(np.float32)
    A21 = np.array(A[2, 1]).astype(np.float32)
    A22 = np.array(A[2, 2]).astype(np.float32)
    A23 = np.array(A[2, 3]).astype(np.float32)
    x = A03
    y = A13
    z = A23
    roll = np.rad2deg(np.arctan2(A21, A22)) 
    pitch = np.rad2deg(np.arctan2(-A20, np.sqrt(A21**2 + A22**2))) 
    yaw = np.rad2deg(np.arctan2(A10, A00)) 
    return np.array([x, y, z, roll, pitch, yaw])
    

def angles_to_follow():
    """

    Returns: Dictionary of the desired angels

    """
    angles = { 't1': [np.deg2rad(0), np.deg2rad(344) ,np.deg2rad(75),np.deg2rad(0), np.deg2rad(300), np.deg2rad(0)],
               't2': [np.deg2rad(0), np.deg2rad(0), np.deg2rad(0), np.deg2rad(0), np.deg2rad(0),np.deg2rad(0)],
               't3': [np.deg2rad(356), np.deg2rad(21), np.deg2rad(150), np.deg2rad(272),np.deg2rad(320) ,np.deg2rad(273) ],
               't4': [np.deg2rad(270),np.deg2rad(148) , np.deg2rad(148), np.deg2rad(270), np.deg2rad(140),np.deg2rad(0) ],
               't5': [np.deg2rad(308), np.deg2rad(88),np.deg2rad(86) ,np.deg2rad(38) , np.deg2rad(358),np.deg2rad(358)],
               't6': [np.deg2rad(335), np.deg2rad(344),np.deg2rad(99) ,np.deg2rad(335) , np.deg2rad(336),np.deg2rad(233)],
               't7': [np.deg2rad(20), np.deg2rad(333),np.deg2rad(6) ,np.deg2rad(244) , np.deg2rad(284),np.deg2rad(322)],
               't8': [np.deg2rad(266), np.deg2rad(312),np.deg2rad(350) ,np.deg2rad(337) , np.deg2rad(252),np.deg2rad(265)],
               't9': [np.deg2rad(255), np.deg2rad(273),np.deg2rad(284) ,np.deg2rad(357) , np.deg2rad(237),np.deg2rad(248)],
               't10': [np.deg2rad(218), np.deg2rad(245),np.deg2rad(218) ,np.deg2rad(58) , np.deg2rad(266),np.deg2rad(318)]}# [radians]
                # 't11': [np.deg2rad(295), np.deg2rad(39) ,np.deg2rad(77),np.deg2rad(280), np.deg2rad(220), np.deg2rad(210)] # part b
    return angles

In [24]:
# For checking
np.set_printoptions(suppress=True)
ang = angles_to_follow()
theta_list = [ang['t1'],ang['t2'],ang['t3'],ang['t4'],ang['t5'],ang['t6'],ang['t7'],ang['t8'],ang['t9'],ang['t10']]
for i in range(10):

    A1=FK(theta_list[i])
    a1 = np.array(A1).astype(np.float32)
    a1 = np.round(a1,3)
    a1 = Array(a1)
    print(f"Actuators Angles {i+1}")
    print(np.rad2deg(theta_list[i]))
    print(f"Homogeneous Matrices {i+1}")
    pprint(a1)
    print(f"Position and Euler Angles {i+1}")
    v1 = xyz_euler(A1)
    v1 = np.round(v1, 2)
    print(v1)

Actuators Angles 1
[  0. 344.  75.   0. 300.   0.]
Homogeneous Matrices 1
⎡-0.866  0.017   0.5    0.439⎤
⎢                            ⎥
⎢ 0.5      0    0.866   0.194⎥
⎢                            ⎥
⎢0.015    1.0   -0.009  0.449⎥
⎢                            ⎥
⎣  0       0      0      1.0 ⎦
Position and Euler Angles 1
[  0.44   0.19   0.45  90.5   -0.87 150.  ]
Actuators Angles 2
[0. 0. 0. 0. 0. 0.]
Homogeneous Matrices 2
⎡ 0   -1.0   0   0.057⎤
⎢                     ⎥
⎢1.0   0     0   -0.01⎥
⎢                     ⎥
⎢ 0    0    1.0  1.003⎥
⎢                     ⎥
⎣ 0    0     0    1.0 ⎦
Position and Euler Angles 2
[ 0.06 -0.01  1.    0.   -0.   90.  ]
Actuators Angles 3
[356.  21. 150. 272. 320. 273.]
Homogeneous Matrices 3
⎡-0.143  -0.971  0.192   0.129 ⎤
⎢                              ⎥
⎢-0.989  0.147   0.009   -0.071⎥
⎢                              ⎥
⎢-0.037  -0.189  -0.981  0.118 ⎥
⎢                              ⎥
⎣  0       0       0      1.0  ⎦
Position and Euler Angles 3
[   0.13

In [ ]:
# plot part a
'''  
Actuators Angles 1
[  0. 344.  75.   0. 300.   0.]
Homogeneous Matrices 1
⎡-0.866  0.017   0.5    0.439⎤
⎢                            ⎥
⎢ 0.5      0    0.866   0.194⎥
⎢                            ⎥
⎢0.015    1.0   -0.009  0.449⎥
⎢                            ⎥
⎣  0       0      0      1.0 ⎦
Position and Euler Angles 1
[  0.44   0.19   0.45  90.5   -0.87 150.  ]
Actuators Angles 2
[0. 0. 0. 0. 0. 0.]
Homogeneous Matrices 2
⎡ 0   -1.0   0   0.057⎤
⎢                     ⎥
⎢1.0   0     0   -0.01⎥
⎢                     ⎥
⎢ 0    0    1.0  1.003⎥
⎢                     ⎥
⎣ 0    0     0    1.0 ⎦
Position and Euler Angles 2
[ 0.06 -0.01  1.    0.   -0.   90.  ]
Actuators Angles 3
[356.  21. 150. 272. 320. 273.]
Homogeneous Matrices 3
⎡-0.143  -0.971  0.192   0.129 ⎤
⎢                              ⎥
⎢-0.989  0.147   0.009   -0.071⎥
⎢                              ⎥
⎢-0.037  -0.189  -0.981  0.118 ⎥
⎢                              ⎥
⎣  0       0       0      1.0  ⎦
Position and Euler Angles 3
[   0.13   -0.07    0.12 -169.11    2.12  -98.21]
Actuators Angles 4
[270. 148. 148. 270. 140.   0.]
Homogeneous Matrices 4
⎡  0    1.0    0     -0.067⎤
⎢                          ⎥
⎢0.766   0   0.643   0.299 ⎥
⎢                          ⎥
⎢0.643   0   -0.766  0.071 ⎥
⎢                          ⎥
⎣  0     0     0      1.0  ⎦
Position and Euler Angles 4
[ -0.07   0.3    0.07 180.   -40.    90.  ]
Actuators Angles 5
[308.  88.  86.  38. 358. 358.]
Homogeneous Matrices 5
⎡0.276   -0.961  -0.013  -0.133⎤
⎢                              ⎥
⎢0.959   0.275   0.061   0.222 ⎥
⎢                              ⎥
⎢-0.055  -0.029  0.998   0.734 ⎥
⎢                              ⎥
⎣  0       0       0      1.0  ⎦
Position and Euler Angles 5
[-0.13  0.22  0.73 -1.69  3.17 73.93]
Actuators Angles 6
[335. 344.  99. 335. 336. 233.]
Homogeneous Matrices 6
⎡-0.34   -0.422   0.84   0.434 ⎤
⎢                              ⎥
⎢-0.763  0.646   0.015   -0.145⎥
⎢                              ⎥
⎢-0.549  -0.636  -0.542  0.235 ⎥
⎢                              ⎥
⎣  0       0       0      1.0  ⎦
Position and Euler Angles 6
[   0.43   -0.14    0.23 -130.41   33.29 -114.03]
Actuators Angles 7
[ 20. 333.   6. 244. 284. 322.]
Homogeneous Matrices 7
⎡-0.251  -0.148  0.957   0.471⎤
⎢                             ⎥
⎢-0.769   0.63   -0.104    0  ⎥
⎢                             ⎥
⎢-0.588  -0.762  -0.272  0.648⎥
⎢                             ⎥
⎣  0       0       0      1.0 ⎦
Position and Euler Angles 7
[   0.47   -0.      0.65 -109.65   35.99 -108.08]
Actuators Angles 8
[266. 312. 350. 337. 252. 265.]
Homogeneous Matrices 8
⎡-0.418  -0.274  0.866   0.143 ⎤
⎢                              ⎥
⎢-0.755  0.636   -0.163  -0.435⎥
⎢                              ⎥
⎢-0.506  -0.722  -0.472   0.48 ⎥
⎢                              ⎥
⎣  0       0       0      1.0  ⎦
Position and Euler Angles 8
[   0.14   -0.44    0.48 -123.19   30.38 -118.99]
Actuators Angles 9
[255. 273. 284. 357. 237. 248.]
Homogeneous Matrices 9
⎡-0.104  -0.556  0.825   0.082 ⎤
⎢                              ⎥
⎢-0.986  -0.051  -0.158  -0.403⎥
⎢                              ⎥
⎢ 0.13   -0.83   -0.543   0.36 ⎥
⎢                              ⎥
⎣  0       0       0      1.0  ⎦
Position and Euler Angles 9
[   0.08   -0.4     0.36 -123.2    -7.45  -95.99]
Actuators Angles 10
[218. 245. 218.  58. 266. 318.]
Homogeneous Matrices 10
⎡-0.213  -0.393  0.894    0.1  ⎤
⎢                              ⎥
⎢-0.851  0.524   0.028   -0.128⎥
⎢                              ⎥
⎢-0.48   -0.756  -0.446  0.252 ⎥
⎢                              ⎥
⎣  0       0       0      1.0  ⎦
Position and Euler Angles 10
[   0.1    -0.13    0.25 -120.57   28.66 -104.02]

'''

In [ ]:
#part b
'''
Actuators Angles
[295.  39.  77. 280. 220. 210.]
Homogeneous Matrices T_ee
⎡0.049   -0.992  0.113   -0.041⎤
⎢                              ⎥
⎢-0.999  -0.047  0.023   -0.007⎥
⎢                              ⎥
⎢-0.017  -0.114  -0.993  0.414 ⎥
⎢                              ⎥
⎣  0       0       0      1.0  ⎦
Homogeneous Matrices T_ee*
⎡0  1  0   0.41⎤
⎢              ⎥
⎢1  0  0    0  ⎥
⎢              ⎥
⎢0  0  -1   0  ⎥
⎢              ⎥
⎣0  0  0    1  ⎦
Position and Euler Angles
[  -0.04   -0.01    0.41 -173.48    0.98  -87.16]
Position and Euler Angles *
[0.41, 0, 0, 180, 0, 90]
'''